In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%load_ext Cython

### Using Cython definition files

In [3]:
%%file mathlib.pxd
cdef int max(int a, int b)
cdef int min(int a, int b)

Writing mathlib.pxd


In [4]:
%%file mathlib.pyx
cdef int max(int a, int b): 
    return a if a > b else b 

cdef int min(int a, int b): 
    return a if a < b else b

Writing mathlib.pyx


In [5]:
%%file setup.py
from distutils.core import setup 
from Cython.Build import cythonize 

setup(name =' Hello', 
      ext_modules = cythonize('mathlib.pyx'))

Writing setup.py


In [6]:
! python setup.py -q build_ext --inplace

Compiling mathlib.pyx because it changed.
Cythonizing mathlib.pyx
mathlib.c:1281:28: warning: unused function '__Pyx_PyObject_AsString'
      [-Wunused-function]
static CYTHON_INLINE char* __Pyx_PyObject_AsString(PyObject* o) {
                           ^
mathlib.c:1278:32: warning: unused function '__Pyx_PyUnicode_FromString'
      [-Wunused-function]
static CYTHON_INLINE PyObject* __Pyx_PyUnicode_FromString(const char* c_str) {
                               ^
mathlib.c:314:29: warning: unused function '__Pyx_Py_UNICODE_strlen'
      [-Wunused-function]
static CYTHON_INLINE size_t __Pyx_Py_UNICODE_strlen(const Py_UNICODE *u)
                            ^
mathlib.c:1343:26: warning: unused function '__Pyx_PyObject_IsTrue'
      [-Wunused-function]
static CYTHON_INLINE int __Pyx_PyObject_IsTrue(PyObject* x) {
                         ^
mathlib.c:1393:33: warning: unused function '__Pyx_PyIndex_AsSsize_t'
      [-Wunused-function]
static CYTHON_INLINE Py_ssize_t __Pyx_PyIndex_AsSsize_t

In [7]:
%%cython
from mathlib cimport max

def chebyshev(int x1, int y1, int x2, int y2): 
    return max(abs( x1 - x2), abs( y1 - y2))

print(chebyshev(1,2,3,4))

2


### Using Rmathlib standalone C libraries

In [12]:
%%file rmath.pyx
#cython: boundscheck=False
#cython: wraparound=False
cimport cython_rmath as r
import numpy as np
cimport numpy as np

cpdef rnorm(int n, double mu=0.0, double sigma=1.0):
    cdef int i
    cdef double[:] xs = np.zeros(n)
    for i in range(n):
        xs[i] = r.rnorm(mu, sigma)
    return np.array(xs)

Overwriting rmath.pyx


In [13]:
%%file setup.py
from distutils.core import setup
from Cython.Distutils import Extension
from Cython.Distutils import build_ext
import cython_rmath

ext = Extension("rmath",
                ["rmath.pyx"],
                define_macros=[('MATHLIB_STANDALONE', '1')],
                include_dirs = ['.', '/Users/cliburn/anaconda/lib/python2.7/site-packages/Cython/Includes',
                                '/Users/cliburn/anaconda/lib/python2.7/site-packages/numpy/core/include'],
                library_dirs = ['.'],
                libraries = ['Rmath', 'm',]
               )

setup(cmdclass = {'build_ext': build_ext}, 
                  ext_modules = [ext])

Overwriting setup.py


In [2]:
! python setup.py --quiet build_ext --inplace

In [3]:
import rmath

rmath.rnorm(10)

array([ 0.26394576,  1.6319731 ,  2.83101399, -0.17998554, -0.7914555 ,
       -1.09934755, -0.55847989,  0.32487065, -0.30605849,  0.72439862])